In [1]:
import jaxrl_m.envs
import gym
import numpy as np
import torch

In [2]:
env = gym.make("sort-mode0-v0", disp=True, debug=False)

2024-03-26 17:14:52.289679: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 17:14:52.315371: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 17:14:52.732628: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/max/anaconda3/envs/offline/lib/python3.8/site-packages/transforms3d/quaternions.py:26: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior an

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 525.105.17
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 525.105.17
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
text argument:dependencies/ravens/ravens/environments/assets
int args: [

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# np array with shape (frames, height, width, channels)
def create_video(ims, actions):
    video = np.array(ims)

    fig = plt.figure()
    im = plt.imshow(video[0,:,:,:])

    plt.close() # this is required to not display the generated image

    def init():
        im.set_data(video[0,:,:,:])

    def animate(i):
        im.set_data(video[i,:,:,:])
        im.axes.set_title(f"Action {i}: {actions[i]}")
        return im

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                                interval=500)
    return anim

ven = NVIDIA Corporation


In [4]:
from jaxrl_m.learners.d4rl_utils import new_get_trj_idx
dataset = env.get_dataset("/home/max/Distributional-Preference-Learning/vpl/jaxrl_m/envs/dataset/sort_0_no_ee_debug.hdf5")
dataset['dones_float'] = dataset["terminals"].astype(np.float32)
trj_idx = new_get_trj_idx(dataset)

def get_traj(i):
    if len(dataset["observations"].shape) == 2:
        traj = {
            "observations": dataset["observations"],
            "actions": dataset["actions"],
            "rewards": dataset["rewards"],
            "dones": dataset["dones_float"],    
        }
    else:
        traj = {
            "observations": dataset["observations"][trj_idx[i][0]:trj_idx[i][1]+1],
            "actions": dataset["actions"][trj_idx[i][0]:trj_idx[i][1]+1],
            "rewards": dataset["rewards"][trj_idx[i][0]:trj_idx[i][1]+1],
            "dones": dataset["dones_float"][trj_idx[i][0]:trj_idx[i][1]+1],
        }
    return traj

Getting trj idx: 100%|██████████| 3/3 [00:00<00:00, 115439.56it/s]


In [5]:
import pybullet as p

def vis_trj(i):
    traj = get_traj(i)
    return visualise_trajectory(env, traj)

def reset_scene(env, traj):
    if traj["observations"][0].shape[0] == 19:
        init_obs = traj["observations"][0][7:]
    else:
        init_obs = traj["observations"][0]
    init_pose_obs = []
    for i in range(4):
        o = init_obs[i*3: (i+1)*3]
        init_pose_obs.append(o[:3])
    env.unwrapped.env.task.init_pose(init_pose_obs)
    # env.reset_end_effector(traj["observations"][0, :7])
    env.reset()
    return env.get_obs()

def visualise_trajectory(env, traj=None, model=None, use_seq=False):
    # print(model)
    assert traj or model
    if traj is None:
        obs = env.reset()
        MAX_LEN = 10
    else:
        obs = reset_scene(env, traj)
        MAX_LEN = len(traj["observations"])

    done = False
    t = 0
    ims = []
    actions = []
    o_a = {
        "observations": [],
        "actions": [],
        "errs": []
    }
    
    while not done and t < MAX_LEN:
        if use_seq:
            obs = np.ones(obs.shape)*t
        if model:
            print("Using model")
            # obs_ = torch.tensor(traj["observations"][t], dtype=torch.float32)
            action = model(torch.tensor(obs, dtype=torch.float32)).detach().numpy()
            
            err = np.linalg.norm(action - traj["actions"][t])
        else:
            action = traj["actions"][t]
            err = 0.0
        o_a["observations"].append(obs)
        o_a["actions"].append(action)
        o_a["errs"].append(err)
        obs, _, done, _ = env.step(action)
        actions.append(action)
        t+=1
        ims.append(env.render(mode="rgb_array"))
    env.unwrapped.env.step(None)
    o_a["observations"] = np.array(o_a["observations"])
    o_a["actions"] = np.array(o_a["actions"])
    o_a["errs"] = np.array(o_a["errs"])
    return create_video(ims, actions), o_a

oracle_trajectory = get_traj(0)

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, observations, actions):
        self.observations = observations
        self.actions = actions

    def __len__(self):
        return len(self.observations)

    def __getitem__(self, idx):
        observation = self.observations[idx]
        action = self.actions[idx]

        return observation, action

# import wandb
def get_model(use_seq=False, traj=None, N=1000, num_samples=None):
    # wandb.init(project="ravens bc", config={"use_seq": use_seq, "num_samples": num_samples, "traj": traj is not None, "N": N})
    if traj is None:    
        actions = dataset["actions"]
        observations = dataset["observations"]
        if num_samples:
            observations = observations[:num_samples]
            actions = actions[:num_samples]
            train_observations, eval_observations, train_actions, eval_actions = observations, observations, actions, actions
        else:
        # # Split the dataset into train and eval sets
            train_observations, eval_observations, train_actions, eval_actions = train_test_split(observations, actions, test_size=0.2, random_state=42)
    else:
        len_traj = len(traj["observations"])
        if use_seq:
            observations = np.array([np.ones_like(traj['observations'][i])*i for i in range(len_traj)], dtype=np.float32)
        else:
            observations = np.array([traj['observations'][i] for i in range(len(traj))], dtype=np.float32)
        actions = np.array([traj['actions'][i] for i in range(len_traj)], dtype=np.float32)
        train_observations, eval_observations, train_actions, eval_actions = observations, observations, actions, actions

    # Create train and eval datasets
    train_dataset = CustomDataset(train_observations, train_actions)
    eval_dataset = CustomDataset(eval_observations, eval_actions)

    # Create dataloaders for train and eval datasets
    batch_size = 256
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

    # Create a 2-layer MLP for supervised learning
    class MLP(torch.nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super(MLP, self).__init__()
            self.model = nn.Sequential(
                nn.Linear(input_size, hidden_size),
                nn.LeakyReLU(),
                nn.Linear(hidden_size, hidden_size),
                nn.LeakyReLU(),
                nn.Linear(hidden_size, hidden_size),
                nn.LeakyReLU(),
                nn.Linear(hidden_size, output_size),
            )
            self.activation = torch.nn.Tanh()

        def forward(self, x):
            x = self.model(x)
            # return self.activation(x)
            return x

    # Define the input size, hidden size, and output size for the MLP
    input_size = len(observations[0])
    hidden_size = 256
    output_size = len(actions[0])

    # Create an instance of the MLP
    mlp = MLP(input_size, hidden_size, output_size)

    # Print the MLP architecture
    # print(mlp)

    # Define the loss function and optimizer
    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)

    # Training loop
    num_epochs = N
    print_every = N//10
    train_losses = []
    eval_losses = []
    prev_loss = np.inf
    for epoch in range(num_epochs):
        train_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(train_dataloader):
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = mlp(inputs)

            # Compute the loss
            loss = loss_fn(outputs, targets)

            # Backward pass
            loss.backward()

            # Update the weights
            optimizer.step()

            # Accumulate the training loss
            train_loss += loss.item()
            # print("inputs", inputs)
            # print("targets", targets)
            # print("outputs", outputs)
            # print("\n\n")

        # Compute the average training loss for the epoch
        train_loss /= len(train_dataloader)

        # Evaluate the model on the eval dataset
        eval_loss = 0.0
        with torch.no_grad():
            for inputs, targets in eval_dataloader:
                outputs = mlp(inputs)
                loss = loss_fn(outputs, targets)
                eval_loss += loss.item()

        # Compute the average eval loss for the epoch
        eval_loss /= len(eval_dataloader)
        eval_losses.append(eval_loss)
        wandb.log({"train_loss": train_loss, "eval_loss": eval_loss})
        if eval_loss < prev_loss:
            prev_loss = eval_loss
            # torch.save(mlp, "best_model.pt")

        # Print the training and eval loss for the epoch
        if (epoch+1)%100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss}, Eval Loss: {eval_loss}")
    return mlp, observations, actions


ven = NVIDIA Corporation


In [7]:
#ORACLE_TRAJ
anim, traj_or = visualise_trajectory(env, oracle_trajectory)
print(traj_or["errs"])
# HTML(anim.to_html5_video())

Reward: 0.45823588342113275
Reward: 1.6528914208573666


IndexError: list index out of range

In [ ]:
#SEQ_TRAJ
mlp_seq, observations, actions  = get_model(use_seq=True, traj=oracle_trajectory, N=1000)
anim, traj_seq = visualise_trajectory(env, oracle_trajectory, mlp_seq, use_seq=True)
# print("errs", traj_seq["errs"])
# print("obs", traj_seq["observations"], observations)
# HTML(anim.to_html5_video())

Epoch 100/1000: Train Loss: 3.836647465504939e-06, Eval Loss: 1.0707769433793146e-05
Epoch 200/1000: Train Loss: 2.949142299257801e-06, Eval Loss: 1.5700422295594763e-07
Epoch 300/1000: Train Loss: 0.0001476532779633999, Eval Loss: 0.00031099715852178633
Epoch 400/1000: Train Loss: 3.834632611443567e-09, Eval Loss: 1.7677090902168402e-09
Epoch 500/1000: Train Loss: 1.844058217757849e-13, Eval Loss: 2.351303830458451e-13
Epoch 600/1000: Train Loss: 6.226331606740132e-06, Eval Loss: 1.3149052392691374e-05
Epoch 700/1000: Train Loss: 4.877179193840675e-08, Eval Loss: 2.2784726994018456e-08
Epoch 800/1000: Train Loss: 8.470032401147742e-13, Eval Loss: 1.1696090060006603e-12
Epoch 900/1000: Train Loss: 6.503423423702068e-14, Eval Loss: 9.691675596551399e-15
Epoch 1000/1000: Train Loss: 3.801296842402546e-14, Eval Loss: 8.779529532575766e-15
Using model
Using model
Using model
Using model


In [ ]:
#BC_TRAJ
# mlp_bc, observations, actions  = get_model(False, oracle_trajectory, N=1000)
anim, traj_bc = visualise_trajectory(env, oracle_trajectory, mlp_bc, False)
HTML(anim.to_html5_video())

Using model
Using model
Using model
Using model


Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/home/max/anaconda3/envs/offline/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/max/anaconda3/envs/offline/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/max/anaconda3/envs/offline/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 286, in check_stop_status
    self._loop_check_status(
  File "/home/max/anaconda3/envs/offline/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 224, in _loop_check_status
    local_handle = request()
  File "/home/max/anaconda3/envs/offline/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 784, in deliver_stop_status
    return self._deliver_stop_status(status)
  File "/home/max/anaconda3/envs/offline/lib/python3.8/site-packages/wandb/sdk/interface/interface_shared.py", line 484, in _deliver_stop_status
    return self._deliver_record(record)
  File "/

: 

In [ ]:
#FULL BC
# mlp_bc, observations, actions  = get_model(False, N=5000, num_samples=1000)
anim, traj_bc = visualise_trajectory(env, traj=get_traj(50), model=mlp_bc)
HTML(anim.to_html5_video())

Using model
Using model
Using model
Using model


In [ ]:
traj_bc["errs"]
traj = get_traj(0)

In [ ]:
traj_bc["observations"][0] - traj["observations"][0][7:], traj_bc["actions"][0] - traj["actions"][0]

(array([0., 0., 0., 0., 0.]),
 array([-0.0316838 , -0.20328444, -0.01308642, -0.02210343, -0.44546914,
         0.00053805], dtype=float32))

In [ ]:
traj_bc["observations"][1] - traj["observations"][1][7:], traj_bc["actions"][1] - traj["actions"][1]

(array([-1.76100027e-06,  5.59890646e-09,  3.84736665e-08, -3.55485497e-09,
        -2.09780793e-10]),
 array([-0.15883514, -0.12714976, -0.00257741, -0.0377436 , -0.2664072 ,
         0.01129122], dtype=float32))

In [ ]:
model_actions = mlp_bc(torch.tensor(traj["observations"][1][7:], dtype=torch.float32)).detach().numpy()
model_actions_bc = mlp_bc(torch.tensor(traj_bc["observations"][1], dtype=torch.float32)).detach().numpy()

In [ ]:
traj_bc["observations"][1] - traj["observations"][1][7:], model_actions - model_actions_bc, model_actions - traj["actions"][1], model_actions_bc - traj["actions"][1]

(array([-1.76100027e-06,  5.59890646e-09,  3.84736665e-08, -3.55485497e-09,
        -2.09780793e-10]),
 array([-1.7881393e-07,  1.4901161e-08,  0.0000000e+00, -1.7881393e-07,
        -1.4901161e-07,  4.0978193e-08], dtype=float32),
 array([-0.15883532, -0.12714975, -0.00257741, -0.03774378, -0.26640734,
         0.01129126], dtype=float32),
 array([-0.15883514, -0.12714976, -0.00257741, -0.0377436 , -0.2664072 ,
         0.01129122], dtype=float32))